# (F) summer change detection and classification
### This program is meant to be call once a year on the 31 octobre
### This program need A (active site definition), B (for VH smoothed profile), C and E to be ran prior its running

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

In [ ]:
def summer_change_classification(annee, dteExe):
    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')
    
    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Ouvre la connexion à la DB
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None

    # Liste de dictionnaires 'sar_id_segment'
    sar_id_segments = []

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

        ## Creation de la table résultat

        table_name = 'classification_chg_summer'

        # strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)
        # cur.execute(strSQL)
        # conn.commit()

        strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (ID_Segment TEXT NOT NULL, dteExe DATE, periode TEXT, NDVI_amplitude NUMERIC(4,3), BAI_amplitude NUMERIC, BI_amplitude NUMERIC, BI2_amplitude NUMERIC, SBI_amplitude NUMERIC, COMPO_amplitude NUMERIC, vegetation INTEGER, building INTEGER, soil INTEGER, UNIQUE(ID_Segment, periode));'.format(table_name)
        cur.execute(strSQL)

        conn.commit()

        # Liste des sites pour lesquels on a des mesures dans l'intervalle donné (de mai première année à août dernière année)
        cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats WHERE acq_date BETWEEN \'{0}-05-01\' AND \'{1}-08-31\' ORDER BY sar_id_segment;'.format(annee-1, annee))
        SARs = cur.fetchall()

        # Pour chaque site et indice concernés et chaque année de notre intervalle, relève les différences de moyenne estivale
        for SAR in SARs:
            site = SAR['sar_id_segment']

            for indice in ['NDVI', 'BAI', 'BI', 'BI2', 'SBI', 'COMPO']:

                table_smooth = '{0}_{1}_{2}_smoothed'.format(site, indice, dteExe)

                # Vérifie si la table moyenne lissée par site et indice existe
                strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                cur.execute(strSQL)

                # Si on a des indices "smoothés", on en demande la valeur moyenne
                INDICE_EXISTS = cur.fetchone()[0]

                if INDICE_EXISTS == True:                           
                    strSQL = '''SELECT CONCAT(a.annee, \'-\', p.annee) AS periode, a.moyenne, p.moyenne AS moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg 
                                FROM (SELECT EXTRACT(YEAR FROM dte) AS annee, AVG(indice) AS moyenne FROM "{0}" WHERE EXTRACT(MONTH FROM dte) BETWEEN 5 AND 8 GROUP BY annee) a, 
                                (SELECT EXTRACT(YEAR FROM dte) AS annee, AVG(indice) AS moyenne FROM "{0}" WHERE EXTRACT(MONTH FROM dte) BETWEEN 5 AND 8 GROUP BY annee) p 
                                WHERE p.annee = a.annee - 1 AND a.annee = {1};'''.format(table_smooth, annee)
                    cur2.execute(strSQL)            
                    lignes = cur2.fetchall()

                    for ligne in lignes:
                        periode = ligne['periode']
                        moyenne_chg = ligne['moyenne_chg']

                        strSQL = '''INSERT INTO "{0}" (ID_Segment, dteExe, periode, {1}_amplitude) VALUES (\'{2}\', \'{3}\', \'{4}\', \'{5}\') 
                                    ON CONFLICT (ID_Segment, periode) DO UPDATE SET {1}_amplitude = \'{5}\', dteExe = \'{3}\';'''.format(table_name, indice, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', periode, moyenne_chg)
                        cur2.execute(strSQL)

            # Enregistre les amplitudes
            conn.commit()


            # Compare les amplitudes de changement des indices aux seuils pour définir les changements
            strSQL = 'SELECT * FROM "{0}" WHERE ID_Segment = \'{1}\' AND dteExe = \'{2}\';'.format(table_name, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}')
            cur.execute(strSQL)
            lignes = cur.fetchall()

            for ligne in lignes:
                periode = ligne['periode']
                NDVI_amplitude = ligne['ndvi_amplitude']
                BAI_amplitude = ligne['bai_amplitude']
                BI_amplitude = ligne['bi_amplitude']
                BI2_amplitude = ligne['bi2_amplitude']
                SBI_amplitude = ligne['sbi_amplitude']

                # changements de l'indice NDVI
                if NDVI_amplitude != None:
                    if NDVI_amplitude >= 0.2:
                        vegetation = 2  # « Augmentation végétation »

                    elif NDVI_amplitude >= 0.1:
                        vegetation = 5  # « Augmentation végétation probable »

                    elif NDVI_amplitude <= -0.2:
                        vegetation = 3  # « Diminution végétation »

                    elif NDVI_amplitude <= -0.1:
                        vegetation = 6  # « Diminution végétation probable »

                    else:
                        vegetation = 0  # « Pas de changement végétation »

                # changements de l'indice BAI
                if BAI_amplitude != None:

                    if abs(BAI_amplitude) >= 0.1:
                        soil = 1  # « Changement de sol »

                    elif abs(BAI_amplitude) >= 0.05:
                        soil = 4  # «  Changement de sol probable »

                    else:
                        soil = 0  # « Pas de changement sol »

                # changements des moyennes des 3 indices BI & BI2 & SBI
                if None not in [BI_amplitude, BI2_amplitude, SBI_amplitude]:

                    if abs(BI_amplitude) >= 250 or abs(BI2_amplitude) >= 250 or abs(SBI_amplitude) >= 400:
                        building = 1  # « Changement de bâtiment »

                    elif abs(BI_amplitude) >= 150 or abs(BI2_amplitude) >= 150 or abs(SBI_amplitude) >= 250:
                        building = 4  # « Changement de bâtiment probable »

                    else :
                        building = 0      # « Pas de changement bâtiments »

                # Enregistre les qualifications
                strSQL = 'UPDATE "{0}" SET vegetation = {1}, soil = {2}, building = {3} WHERE ID_Segment = \'{4}\' AND dteExe = \'{5}\' AND periode = \'{6}\';'.format(table_name, vegetation, soil, building, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', periode)
                cur2.execute(strSQL)

                # Enregistre les updates
                conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if cur2 is not None:
            cur2.close()

        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()